# 🔍 Embeddings y Similitud en Datos

Objetivo: usar embeddings para búsqueda semántica en catálogos de datos, detección de duplicados, clustering, y recomendaciones.

- Duración: 90 min
- Dificultad: Media
- Stack: OpenAI Embeddings, scikit-learn, FAISS

## 📊 Embeddings: Arquitectura y Fundamentos

Los **embeddings** son representaciones vectoriales densas de datos (texto, imágenes, audio) en espacios de dimensión reducida que capturan **relaciones semánticas**. En Data Engineering, permiten búsqueda semántica, deduplicación inteligente, clustering y recomendaciones en catálogos de datos.

### 🏗️ Evolución de Embeddings

```
2013: Word2Vec (Google)          → Embeddings de palabras con CBOW/Skip-gram
2017: Sentence-BERT               → Embeddings de frases con transformers
2020: OpenAI text-embedding-ada   → API embeddings general-purpose
2023: text-embedding-3-small/large → Mejor calidad + dimensionalidad variable
2024: Matryoshka Embeddings       → Truncar dimensiones sin perder mucha calidad
```

### 📐 Arquitectura de Modelos de Embeddings

```
┌─────────────────────────────────────────────────────────────┐
│                    MODELO DE EMBEDDINGS                      │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  Input Text                                                  │
│  "Tabla de ventas con transacciones diarias"                │
│        ↓                                                     │
│  ┌─────────────────────────────────────────────────────┐    │
│  │ 1. Tokenización                                     │    │
│  │    - WordPiece / BPE / SentencePiece                │    │
│  │    - Tokens: [101, 5555, 2049, 7231, 102]          │    │
│  └─────────────────────────────────────────────────────┘    │
│        ↓                                                     │
│  ┌─────────────────────────────────────────────────────┐    │
│  │ 2. Transformer Encoder (BERT/RoBERTa/etc)           │    │
│  │    - 12 capas, 768 hidden units                     │    │
│  │    - Self-attention multi-head                      │    │
│  │    - Feed-forward networks                          │    │
│  └─────────────────────────────────────────────────────┘    │
│        ↓                                                     │
│  ┌─────────────────────────────────────────────────────┐    │
│  │ 3. Pooling Strategy                                 │    │
│  │    - [CLS] token: usa primer token (BERT)           │    │
│  │    - Mean pooling: promedio de todos (SBERT) ✓      │    │
│  │    - Max pooling: máximo por dimensión              │    │
│  └─────────────────────────────────────────────────────┘    │
│        ↓                                                     │
│  ┌─────────────────────────────────────────────────────┐    │
│  │ 4. Normalization (L2)                               │    │
│  │    - Vector unitario: ||v|| = 1                     │    │
│  │    - Permite usar dot product = cosine similarity   │    │
│  └─────────────────────────────────────────────────────┘    │
│        ↓                                                     │
│  Output Vector                                               │
│  [0.023, -0.145, 0.089, ..., 0.112]  (1536 dimensiones)    │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### 🎯 Comparación de Modelos de Embeddings

| Modelo | Dimensiones | Precio ($/1M tokens) | MTEB Score | Tokens Max | Caso de Uso |
|--------|-------------|---------------------|------------|------------|-------------|
| **text-embedding-3-small** | 512-1536 | $0.02 | 62.3 | 8191 | 🚀 Costo-efectivo para búsqueda general |
| **text-embedding-3-large** | 1024-3072 | $0.13 | 64.6 | 8191 | ⭐ Máxima calidad OpenAI |
| **text-embedding-ada-002** | 1536 | $0.10 | 61.0 | 8191 | 📦 Legacy, usar v3-small |
| **all-MiniLM-L6-v2** | 384 | Free (OSS) | 58.8 | 256 | 💰 Gratuito, rápido, local |
| **all-mpnet-base-v2** | 768 | Free (OSS) | 63.3 | 384 | 🎯 Mejor OSS general-purpose |
| **bge-large-en-v1.5** | 1024 | Free (OSS) | 63.9 | 512 | 🏆 Estado del arte OSS |
| **voyage-large-2-instruct** | 1024 | $0.12 | 68.3 | 16000 | 🔬 Mejor absoluto + contexto largo |

**MTEB** (Massive Text Embedding Benchmark): evalúa embeddings en 58 tareas (clasificación, clustering, retrieval, etc.). Score >60 es bueno, >65 excelente.

### 🔧 Implementación con Múltiples Modelos

```python
from typing import List, Literal
import numpy as np
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import os

class EmbeddingGenerator:
    """Generador unificado de embeddings con múltiples modelos."""
    
    def __init__(
        self, 
        model: Literal["openai-small", "openai-large", "sbert", "bge"] = "openai-small"
    ):
        self.model_type = model
        
        if model.startswith("openai"):
            self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
            self.model_name = (
                "text-embedding-3-small" if model == "openai-small" 
                else "text-embedding-3-large"
            )
        elif model == "sbert":
            self.model = SentenceTransformer('all-mpnet-base-v2')
        elif model == "bge":
            self.model = SentenceTransformer('BAAI/bge-large-en-v1.5')
    
    def embed(self, texts: List[str], batch_size: int = 100) -> np.ndarray:
        """Genera embeddings con batching para eficiencia."""
        if self.model_type.startswith("openai"):
            embeddings = []
            for i in range(0, len(texts), batch_size):
                batch = texts[i:i+batch_size]
                response = self.client.embeddings.create(
                    model=self.model_name,
                    input=batch
                )
                embeddings.extend([d.embedding for d in response.data])
            return np.array(embeddings)
        else:
            # Sentence Transformers ya hace batching internamente
            return self.model.encode(texts, batch_size=batch_size, show_progress_bar=True)
    
    def embed_single(self, text: str) -> np.ndarray:
        """Embedding de un solo texto."""
        return self.embed([text])[0]
    
    @property
    def dimensions(self) -> int:
        """Retorna dimensionalidad del modelo."""
        if self.model_type == "openai-small":
            return 1536
        elif self.model_type == "openai-large":
            return 3072
        elif self.model_type == "sbert":
            return 768
        elif self.model_type == "bge":
            return 1024

# Ejemplo: comparación de modelos
textos = [
    "Tabla de ventas con transacciones diarias",
    "Data de transacciones de venta por día"
]

for model in ["openai-small", "sbert", "bge"]:
    generator = EmbeddingGenerator(model=model)
    embs = generator.embed(textos)
    
    # Similitud coseno
    from sklearn.metrics.pairwise import cosine_similarity
    sim = cosine_similarity([embs[0]], [embs[1]])[0][0]
    
    print(f"{model:15} | dims={generator.dimensions:4} | similarity={sim:.4f}")

# Salida esperada:
# openai-small    | dims=1536 | similarity=0.9245
# sbert           | dims= 768 | similarity=0.8876
# bge             | dims=1024 | similarity=0.9102
```

### 🎨 Dimensionalidad Variable (Matryoshka Embeddings)

Los modelos `text-embedding-3-*` soportan **truncamiento de dimensiones** sin reentrenar:

```python
# Generar embedding de 1536 dimensiones
response = client.embeddings.create(
    model="text-embedding-3-small",
    input="Tabla de ventas",
    dimensions=1536  # Valor por defecto
)

# Truncar a 512 dimensiones (3x más pequeño, ~5% pérdida de calidad)
response_small = client.embeddings.create(
    model="text-embedding-3-small",
    input="Tabla de ventas",
    dimensions=512
)

# Comparación espacio vs calidad
# 1536 dims: 100% calidad, 6 KB por vector
#  768 dims:  98% calidad, 3 KB por vector ✓ Sweet spot
#  512 dims:  95% calidad, 2 KB por vector (búsqueda rápida)
#  256 dims:  90% calidad, 1 KB por vector (millones de vectores)
```

### 💾 Caché de Embeddings en Producción

Generar embeddings es costoso ($0.02-$0.13 por 1M tokens). **Siempre cachear**:

```python
import sqlite3
import hashlib
import json

class EmbeddingCache:
    """Cache persistente de embeddings con SQLite."""
    
    def __init__(self, db_path: str = "embeddings.db"):
        self.conn = sqlite3.connect(db_path)
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS embeddings (
                text_hash TEXT PRIMARY KEY,
                text TEXT,
                model TEXT,
                embedding BLOB,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        self.conn.commit()
    
    def _hash(self, text: str, model: str) -> str:
        """Hash único para texto + modelo."""
        return hashlib.sha256(f"{model}:{text}".encode()).hexdigest()
    
    def get(self, text: str, model: str) -> np.ndarray | None:
        """Obtiene embedding del cache."""
        h = self._hash(text, model)
        cursor = self.conn.execute(
            "SELECT embedding FROM embeddings WHERE text_hash = ?", (h,)
        )
        row = cursor.fetchone()
        if row:
            return np.frombuffer(row[0], dtype=np.float32)
        return None
    
    def set(self, text: str, model: str, embedding: np.ndarray):
        """Guarda embedding en cache."""
        h = self._hash(text, model)
        self.conn.execute(
            "INSERT OR REPLACE INTO embeddings (text_hash, text, model, embedding) VALUES (?, ?, ?, ?)",
            (h, text, model, embedding.astype(np.float32).tobytes())
        )
        self.conn.commit()
    
    def stats(self) -> dict:
        """Estadísticas del cache."""
        cursor = self.conn.execute("""
            SELECT 
                model,
                COUNT(*) as count,
                MIN(created_at) as oldest,
                MAX(created_at) as newest
            FROM embeddings
            GROUP BY model
        """)
        return {row[0]: {"count": row[1], "oldest": row[2], "newest": row[3]} 
                for row in cursor.fetchall()}

# Uso con cache
cache = EmbeddingCache()
generator = EmbeddingGenerator("openai-small")

def get_embedding(text: str) -> np.ndarray:
    """Obtiene embedding con cache transparente."""
    cached = cache.get(text, "openai-small")
    if cached is not None:
        return cached
    
    # Cache miss: generar y guardar
    emb = generator.embed_single(text)
    cache.set(text, "openai-small", emb)
    return emb

# Estadísticas
print(cache.stats())
# {'openai-small': {'count': 15234, 'oldest': '2024-01-15', 'newest': '2024-02-20'}}
```

### 📊 Optimización de Costos

```python
# Estrategia 1: Batch processing (reduce latencia + overhead)
texts = ["texto 1", "texto 2", ..., "texto 1000"]
embeddings = generator.embed(texts, batch_size=100)  # 10 llamadas API vs 1000

# Estrategia 2: Usar modelo local para desarrollo
dev_generator = EmbeddingGenerator("sbert")  # Gratis
prod_generator = EmbeddingGenerator("openai-large")  # Solo producción

# Estrategia 3: Preprocesar texto para reducir tokens
def clean_text(text: str) -> str:
    """Limpia texto antes de embeddings."""
    import re
    # Remover URLs, emails, exceso de whitespace
    text = re.sub(r'http\S+|www\.\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text[:8000]  # Truncar a límite del modelo

# Estrategia 4: Dimensiones reducidas para búsqueda inicial
# Usar 512 dims para búsqueda rápida (retrieval)
# Luego re-ranking con 1536 dims para top-100 candidatos
```

### 🔬 Casos de Uso en Data Engineering

| Aplicación | Técnica | Ejemplo |
|------------|---------|---------|
| **Búsqueda Semántica** | Cosine similarity + top-k | "ventas último mes" → encuentra `revenue_monthly` |
| **Deduplicación** | Similitud > threshold | "iPhone 13" ≈ "Apple iPhone 13" (0.95 similarity) |
| **Clustering** | K-Means en espacio vectorial | Agrupar tablas por dominio (finanzas, marketing, ops) |
| **Recomendaciones** | KNN en embeddings | Usuario vio pipeline A → recomendar pipeline B similar |
| **Etiquetado Automático** | Clasificación con embeddings | Embedding + classifier → asignar tags a datasets |
| **Detección de Anomalías** | Distancia a centroide | Detectar tablas "raras" en catálogo |

**Ejemplo real**: En el catálogo de datos de Airbnb, usan embeddings de descripciones de tablas para:
- Búsqueda: "host earnings" encuentra `payments.host_payouts`
- Auto-tagging: Clasificar 10,000 tablas en 50 categorías
- Data lineage: Encontrar tablas relacionadas por similitud semántica

## 🎯 Métricas de Similitud: Más Allá del Coseno

La **similitud coseno** es la métrica más común para embeddings, pero existen alternativas con diferentes propiedades matemáticas y casos de uso. Elegir la métrica correcta impacta directamente la calidad de búsqueda, deduplicación y clustering.

### 📐 Comparación de Métricas de Similitud

```
┌──────────────────────────────────────────────────────────────────┐
│                    MÉTRICAS DE SIMILITUD                         │
├──────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Vector A: [0.5, 0.8, 0.3]       Vector B: [0.4, 0.9, 0.2]     │
│                                                                  │
│  1️⃣ COSINE SIMILARITY (ángulo entre vectores)                   │
│     ┌─────────────────────────────────────────────────────┐     │
│     │  cos(θ) = (A·B) / (||A|| × ||B||)                   │     │
│     │         = 0.94 / (1.02 × 1.01)                      │     │
│     │         = 0.912  ✓ Valor [0,1] o [-1,1]            │     │
│     └─────────────────────────────────────────────────────┘     │
│     Propiedades:                                                 │
│     • Ignora magnitud (solo dirección)                          │
│     • Invariante a escala: [1,2,3] ≈ [2,4,6]                   │
│     • Rápido con vectores normalizados: cos = A·B               │
│     Uso: Embeddings de texto (siempre normalizados)             │
│                                                                  │
│  2️⃣ EUCLIDEAN DISTANCE (distancia L2)                           │
│     ┌─────────────────────────────────────────────────────┐     │
│     │  d(A,B) = √(Σ(ai - bi)²)                            │     │
│     │         = √((0.5-0.4)² + (0.8-0.9)² + (0.3-0.2)²)  │     │
│     │         = √(0.01 + 0.01 + 0.01)                     │     │
│     │         = 0.173  ✓ Menor = más similar              │     │
│     └─────────────────────────────────────────────────────┘     │
│     Propiedades:                                                 │
│     • Considera magnitud (tamaño del vector)                    │
│     • Sensible a outliers (por el cuadrado)                     │
│     • Curse of dimensionality en alta dimensión                 │
│     Uso: Embeddings de imágenes, clustering K-Means             │
│                                                                  │
│  3️⃣ DOT PRODUCT (producto punto)                                │
│     ┌─────────────────────────────────────────────────────┐     │
│     │  A·B = Σ(ai × bi)                                    │     │
│     │      = (0.5×0.4) + (0.8×0.9) + (0.3×0.2)            │     │
│     │      = 0.2 + 0.72 + 0.06                            │     │
│     │      = 0.98  ✓ Mayor = más similar                  │     │
│     └─────────────────────────────────────────────────────┘     │
│     Propiedades:                                                 │
│     • Si vectores normalizados: dot = cosine                    │
│     • Captura magnitud + dirección                              │
│     • Más rápido (sin divisiones ni sqrt)                       │
│     Uso: Vectores normalizados, modelos de lenguaje             │
│                                                                  │
│  4️⃣ MANHATTAN DISTANCE (distancia L1)                           │
│     ┌─────────────────────────────────────────────────────┐     │
│     │  d(A,B) = Σ|ai - bi|                                │     │
│     │         = |0.5-0.4| + |0.8-0.9| + |0.3-0.2|         │     │
│     │         = 0.1 + 0.1 + 0.1                           │     │
│     │         = 0.3  ✓ Menor = más similar                │     │
│     └─────────────────────────────────────────────────────┘     │
│     Propiedades:                                                 │
│     • Menos sensible a outliers que L2                          │
│     • Más interpretable (suma de diferencias)                   │
│     • Útil en espacios de alta dimensión                        │
│     Uso: Embeddings sparse, feature vectors                     │
│                                                                  │
└──────────────────────────────────────────────────────────────────┘
```

### 🔬 Implementación Optimizada de Métricas

```python
import numpy as np
from typing import Literal
from sklearn.metrics.pairwise import (
    cosine_similarity, euclidean_distances, manhattan_distances
)
import time

class SimilarityMetrics:
    """Implementación optimizada de métricas de similitud."""
    
    @staticmethod
    def cosine(a: np.ndarray, b: np.ndarray) -> float:
        """
        Similitud coseno: cos(θ) = (A·B) / (||A|| × ||B||)
        Rango: [-1, 1] donde 1 = idénticos, 0 = ortogonales, -1 = opuestos
        """
        # Optimización: si vectores están normalizados, usar dot product
        if np.isclose(np.linalg.norm(a), 1.0) and np.isclose(np.linalg.norm(b), 1.0):
            return np.dot(a, b)
        return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    
    @staticmethod
    def euclidean(a: np.ndarray, b: np.ndarray) -> float:
        """
        Distancia euclidiana: d = √(Σ(ai - bi)²)
        Rango: [0, ∞] donde 0 = idénticos
        """
        return np.linalg.norm(a - b)
    
    @staticmethod
    def manhattan(a: np.ndarray, b: np.ndarray) -> float:
        """
        Distancia Manhattan: d = Σ|ai - bi|
        Rango: [0, ∞] donde 0 = idénticos
        """
        return np.sum(np.abs(a - b))
    
    @staticmethod
    def dot_product(a: np.ndarray, b: np.ndarray) -> float:
        """
        Producto punto: A·B = Σ(ai × bi)
        Si normalizados: equivalente a cosine
        """
        return np.dot(a, b)
    
    @staticmethod
    def batch_similarity(
        queries: np.ndarray, 
        corpus: np.ndarray, 
        metric: Literal["cosine", "euclidean", "manhattan", "dot"] = "cosine"
    ) -> np.ndarray:
        """
        Calcula similitud entre múltiples queries y corpus (vectorizado).
        
        Args:
            queries: (n_queries, dim)
            corpus: (n_corpus, dim)
            metric: tipo de métrica
        
        Returns:
            (n_queries, n_corpus) matriz de similitudes/distancias
        """
        if metric == "cosine":
            return cosine_similarity(queries, corpus)
        elif metric == "euclidean":
            return -euclidean_distances(queries, corpus)  # Negativo para ordenar descendente
        elif metric == "manhattan":
            return -manhattan_distances(queries, corpus)
        elif metric == "dot":
            return queries @ corpus.T
        else:
            raise ValueError(f"Métrica no soportada: {metric}")

# Benchmark de métricas
def benchmark_metrics():
    """Compara velocidad y resultados de diferentes métricas."""
    np.random.seed(42)
    n_vectors = 10000
    dim = 768
    
    # Generar vectores normalizados (simulando embeddings)
    vectors = np.random.randn(n_vectors, dim).astype(np.float32)
    vectors = vectors / np.linalg.norm(vectors, axis=1, keepdims=True)
    
    query = vectors[0]
    
    results = {}
    for metric in ["cosine", "euclidean", "manhattan", "dot"]:
        start = time.perf_counter()
        
        if metric == "cosine":
            scores = cosine_similarity([query], vectors)[0]
        elif metric == "euclidean":
            scores = -euclidean_distances([query], vectors)[0]
        elif metric == "manhattan":
            scores = -manhattan_distances([query], vectors)[0]
        elif metric == "dot":
            scores = vectors @ query
        
        elapsed = (time.perf_counter() - start) * 1000
        top_5_indices = np.argsort(scores)[-6:-1][::-1]  # Excluir el mismo vector
        
        results[metric] = {
            "time_ms": elapsed,
            "top_5": top_5_indices.tolist(),
            "top_scores": scores[top_5_indices].tolist()
        }
    
    return results

# Ejecutar benchmark
bench_results = benchmark_metrics()
for metric, data in bench_results.items():
    print(f"{metric:10} | {data['time_ms']:6.2f} ms | top scores: {data['top_scores'][:3]}")

# Salida esperada:
# cosine     |  15.32 ms | top scores: [0.987, 0.976, 0.965]
# euclidean  |  18.45 ms | top scores: [-0.234, -0.289, -0.312]
# manhattan  |  12.78 ms | top scores: [-8.456, -9.123, -9.678]
# dot        |   3.21 ms | top scores: [0.987, 0.976, 0.965] ✓ Más rápido para normalizados
```

### 🎯 Cuándo Usar Cada Métrica

| Métrica | Ventajas | Desventajas | Caso de Uso Ideal |
|---------|----------|-------------|-------------------|
| **Cosine** | • Ignora magnitud (solo semántica)<br>• Intuitivo: 0-1 o -1 a 1<br>• Estándar en NLP | • Requiere normalización<br>• División costosa | ✅ **Embeddings de texto**<br>Búsqueda semántica, RAG, clasificación |
| **Dot Product** | • Más rápido (sin división)<br>• Equivalente a cosine si normalizado<br>• Eficiente en GPU | • Sensible a magnitud<br>• Requiere normalización previa | ✅ **Producción a escala**<br>FAISS, búsqueda en millones de vectores |
| **Euclidean** | • Considera magnitud<br>• Métrica geométrica natural<br>• K-Means lo usa | • Curse of dimensionality<br>• Outliers impactan mucho | ✅ **Clustering de imágenes**<br>K-Means, espacios de baja dimensión |
| **Manhattan** | • Robusto a outliers<br>• Interpretable (suma diferencias)<br>• Mejor en alta dimensión | • No tan común<br>• Menos intuitivo semánticamente | ✅ **Feature vectors sparse**<br>Embeddings con muchos ceros |

### 🔍 Búsqueda Top-K Optimizada

```python
import heapq
from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class SearchResult:
    """Resultado de búsqueda con score y metadata."""
    index: int
    score: float
    text: str
    metadata: dict

class TopKSearch:
    """Búsqueda top-k con múltiples estrategias de optimización."""
    
    def __init__(self, embeddings: np.ndarray, texts: List[str], metadata: List[dict]):
        """
        Args:
            embeddings: (n, dim) matriz de embeddings normalizados
            texts: lista de textos originales
            metadata: lista de metadatos por cada embedding
        """
        self.embeddings = embeddings
        self.texts = texts
        self.metadata = metadata
        
        # Normalizar embeddings para usar dot product = cosine
        self.embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    def search_vectorized(
        self, 
        query: np.ndarray, 
        top_k: int = 10
    ) -> List[SearchResult]:
        """Búsqueda vectorizada (NumPy) - más rápida para corpus pequeño (<100K)."""
        query = query / np.linalg.norm(query)  # Normalizar query
        
        # Dot product = cosine similarity para vectores normalizados
        scores = self.embeddings @ query  # (n,) array
        
        # Top-k con argpartition (más rápido que argsort para k pequeño)
        if top_k < len(scores):
            top_indices = np.argpartition(scores, -top_k)[-top_k:]
            top_indices = top_indices[np.argsort(scores[top_indices])][::-1]
        else:
            top_indices = np.argsort(scores)[::-1][:top_k]
        
        return [
            SearchResult(
                index=int(idx),
                score=float(scores[idx]),
                text=self.texts[idx],
                metadata=self.metadata[idx]
            )
            for idx in top_indices
        ]
    
    def search_heap(
        self, 
        query: np.ndarray, 
        top_k: int = 10
    ) -> List[SearchResult]:
        """Búsqueda con heap (memoria constante) - útil para corpus muy grande."""
        query = query / np.linalg.norm(query)
        
        # Min-heap de tamaño top_k (mantiene los k mayores)
        heap = []
        for i, emb in enumerate(self.embeddings):
            score = np.dot(emb, query)
            
            if len(heap) < top_k:
                heapq.heappush(heap, (score, i))
            elif score > heap[0][0]:
                heapq.heapreplace(heap, (score, i))
        
        # Ordenar resultados descendente
        results = sorted(heap, reverse=True)
        
        return [
            SearchResult(
                index=idx,
                score=score,
                text=self.texts[idx],
                metadata=self.metadata[idx]
            )
            for score, idx in results
        ]
    
    def search_threshold(
        self, 
        query: np.ndarray, 
        threshold: float = 0.8
    ) -> List[SearchResult]:
        """Búsqueda por threshold (retorna todos los que superan umbral)."""
        query = query / np.linalg.norm(query)
        scores = self.embeddings @ query
        
        # Filtrar por threshold
        above_threshold = np.where(scores >= threshold)[0]
        sorted_indices = above_threshold[np.argsort(scores[above_threshold])][::-1]
        
        return [
            SearchResult(
                index=int(idx),
                score=float(scores[idx]),
                text=self.texts[idx],
                metadata=self.metadata[idx]
            )
            for idx in sorted_indices
        ]

# Ejemplo de uso
texts = [
    "Tabla de ventas con transacciones diarias",
    "Data de transacciones de venta por día",
    "Catálogo de productos con precios",
    "Información demográfica de clientes"
]
embeddings = np.random.randn(len(texts), 768)  # Simular embeddings
metadata = [{"owner": "analytics", "source": f"dwh.table_{i}"} for i in range(len(texts))]

search_engine = TopKSearch(embeddings, texts, metadata)
query_embedding = np.random.randn(768)

# Búsqueda top-3
results = search_engine.search_vectorized(query_embedding, top_k=3)
for r in results:
    print(f"Score {r.score:.3f} | {r.text} | {r.metadata['source']}")

# Búsqueda por threshold (duplicados)
similar = search_engine.search_threshold(embeddings[0], threshold=0.95)
print(f"\nEncontrados {len(similar)} textos muy similares (>0.95)")
```

### 📊 Análisis de Distribución de Similitudes

```python
import matplotlib.pyplot as plt
import seaborn as sns

def analyze_similarity_distribution(embeddings: np.ndarray, sample_size: int = 1000):
    """Analiza la distribución de similitudes en un corpus."""
    np.random.seed(42)
    
    # Normalizar embeddings
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    # Muestrear pares aleatorios
    n = len(embeddings)
    pairs = np.random.choice(n, size=(sample_size, 2), replace=True)
    
    similarities = []
    for i, j in pairs:
        if i != j:
            sim = np.dot(embeddings[i], embeddings[j])
            similarities.append(sim)
    
    similarities = np.array(similarities)
    
    # Estadísticas
    stats = {
        "mean": similarities.mean(),
        "std": similarities.std(),
        "min": similarities.min(),
        "max": similarities.max(),
        "median": np.median(similarities),
        "q95": np.percentile(similarities, 95),  # Threshold para "muy similar"
        "q99": np.percentile(similarities, 99)   # Threshold para "duplicados"
    }
    
    print("Distribución de similitudes en corpus:")
    print(f"  Media: {stats['mean']:.3f} ± {stats['std']:.3f}")
    print(f"  Rango: [{stats['min']:.3f}, {stats['max']:.3f}]")
    print(f"  Percentil 95: {stats['q95']:.3f}  ← Threshold para 'muy similar'")
    print(f"  Percentil 99: {stats['q99']:.3f}  ← Threshold para 'duplicados'")
    
    return stats, similarities

# Ejemplo: analizar corpus de embeddings
embeddings = np.random.randn(10000, 768)
stats, sims = analyze_similarity_distribution(embeddings)

# Salida típica:
# Distribución de similitudes en corpus:
#   Media: 0.012 ± 0.087
#   Rango: [-0.289, 0.354]
#   Percentil 95: 0.156  ← Threshold para 'muy similar'
#   Percentil 99: 0.213  ← Threshold para 'duplicados'
#
# Interpretación:
# - Media cercana a 0: corpus diverso (bueno)
# - Usar threshold 0.95-0.99 para deduplicación
# - Usar threshold 0.80-0.90 para búsqueda de similares
```

### 🎨 Visualización de Similitudes

```python
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_embeddings_2d(
    embeddings: np.ndarray, 
    labels: List[str], 
    method: Literal["pca", "tsne"] = "tsne"
):
    """Visualiza embeddings en 2D para análisis exploratorio."""
    
    # Reducción de dimensionalidad
    if method == "pca":
        reducer = PCA(n_components=2, random_state=42)
    else:  # tsne
        reducer = TSNE(n_components=2, random_state=42, perplexity=30)
    
    coords_2d = reducer.fit_transform(embeddings)
    
    # Plot
    plt.figure(figsize=(12, 8))
    plt.scatter(coords_2d[:, 0], coords_2d[:, 1], alpha=0.6)
    
    # Anotar algunos puntos
    for i, label in enumerate(labels[:20]):  # Primeros 20 para no saturar
        plt.annotate(
            label[:30],  # Truncar texto
            (coords_2d[i, 0], coords_2d[i, 1]),
            fontsize=8,
            alpha=0.7
        )
    
    plt.title(f"Visualización de Embeddings ({method.upper()})")
    plt.xlabel("Dimensión 1")
    plt.ylabel("Dimensión 2")
    plt.tight_layout()
    plt.show()
    
    return coords_2d
```

### 🚀 Reglas de Oro para Métricas de Similitud

1. **Embeddings de texto normalizados**: Usa **dot product** (más rápido que cosine)
2. **Corpus <100K vectores**: Búsqueda **vectorizada con NumPy**
3. **Corpus >1M vectores**: Usar **FAISS** con índices especializados
4. **Deduplicación**: Threshold **>0.95** (percentil 99 del corpus)
5. **Búsqueda semántica**: Threshold **>0.80** (top-k con k=5-10)
6. **Clustering**: **Euclidean distance** con K-Means
7. **Monitorear distribución**: Similitudes extremas (>0.99 o <0.1) son sospechosas

## ⚡ Búsqueda Vectorial a Escala: FAISS y Annoy

Cuando el corpus supera **100K embeddings**, la búsqueda lineal (calcular similitud con todos los vectores) se vuelve prohibitivamente lenta. **FAISS** (Facebook AI Similarity Search) y **Annoy** (Spotify) son librerías especializadas que aceleran la búsqueda usando **Approximate Nearest Neighbors (ANN)** con índices optimizados.

### 🏗️ Arquitectura de FAISS

```
┌─────────────────────────────────────────────────────────────────┐
│                    FAISS: ÍNDICES Y PERFORMANCE                  │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  PROBLEMA: Búsqueda lineal O(n×d) es lenta para n grande       │
│  10M vectores × 768 dims × 4 bytes = 30 GB memoria             │
│  Búsqueda 1 query: 10M dot products = ~500ms                   │
│                                                                  │
│  SOLUCIÓN: Índices especializados con trade-off speed vs accuracy│
│                                                                  │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ 1. IndexFlatL2 / IndexFlatIP (EXACTO)                  │    │
│  │    - Búsqueda exhaustiva (brute force)                 │    │
│  │    - 100% recall, pero lento                           │    │
│  │    - Baseline para comparar                            │    │
│  │    Complejidad: O(n×d)                                 │    │
│  │    Uso: <100K vectores, validación                     │    │
│  └─────────────────────────────────────────────────────────┘    │
│        ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ 2. IndexIVFFlat (PARTICIONADO)                         │    │
│  │    ┌─────────────────────────────────────────┐         │    │
│  │    │ a) Training: K-Means agrupa en nlist    │         │    │
│  │    │    centroides (ej. nlist=1000)          │         │    │
│  │    │                                          │         │    │
│  │    │ b) Indexing: asigna cada vector al      │         │    │
│  │    │    centroide más cercano                │         │    │
│  │    │                                          │         │    │
│  │    │ c) Search: busca en nprobe clusters     │         │    │
│  │    │    más cercanos (ej. nprobe=10)         │         │    │
│  │    └─────────────────────────────────────────┘         │    │
│  │    Complejidad: O(nprobe × n/nlist × d)               │    │
│  │    Recall: 90-95% con nprobe=10                       │    │
│  │    Speedup: 10-100x vs flat                           │    │
│  │    Uso: 100K - 10M vectores                           │    │
│  └─────────────────────────────────────────────────────────┘    │
│        ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ 3. IndexIVFPQ (COMPRESIÓN + PARTICIONADO)             │    │
│  │    - Product Quantization (PQ): comprime vectores      │    │
│  │      768 dims → 96 bytes (8x compresión)               │    │
│  │    - Divide vector en m subvectores                    │    │
│  │    - Cada subvector → codebook de 256 centroides      │    │
│  │    - Búsqueda en espacio comprimido (rápida)          │    │
│  │    Recall: 85-90%                                      │    │
│  │    Speedup: 100-1000x vs flat                         │    │
│  │    Memory: 10-20x menos que IVFFlat                   │    │
│  │    Uso: 10M - 1B vectores                             │    │
│  └─────────────────────────────────────────────────────────┘    │
│        ↓                                                         │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │ 4. IndexHNSW (GRAFO JERÁRQUICO)                        │    │
│  │    - Hierarchical Navigable Small World                │    │
│  │    - Grafo multi-nivel con conexiones                  │    │
│  │    - Búsqueda greedy navegando grafo                   │    │
│  │    Recall: 95-99% (mejor que IVF)                      │    │
│  │    Speedup: 50-500x vs flat                            │    │
│  │    Memory: Similar a flat (sin compresión)             │    │
│  │    Uso: <10M vectores, alta precisión requerida       │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### 🔧 Implementación de FAISS para Diferentes Escalas

```python
import faiss
import numpy as np
from typing import Tuple, List
import time

class FAISSSearchEngine:
    """Motor de búsqueda vectorial con FAISS para diferentes escalas."""
    
    def __init__(
        self, 
        dimension: int, 
        index_type: str = "auto",
        metric: str = "cosine"
    ):
        """
        Args:
            dimension: dimensionalidad de embeddings
            index_type: "flat", "ivf", "ivfpq", "hnsw", "auto"
            metric: "cosine" (IP para normalizados) o "l2" (euclidean)
        """
        self.dimension = dimension
        self.index_type = index_type
        self.metric = metric
        self.index = None
        self.n_vectors = 0
    
    def build_index(self, embeddings: np.ndarray, nlist: int = None):
        """
        Construye índice FAISS apropiado según tamaño del corpus.
        
        Args:
            embeddings: (n, dim) matriz de embeddings
            nlist: número de clusters para IVF (auto si None)
        """
        self.n_vectors = len(embeddings)
        embeddings = embeddings.astype('float32')
        
        # Normalizar si métrica es cosine
        if self.metric == "cosine":
            faiss.normalize_L2(embeddings)  # In-place normalization
        
        # Selección automática de índice según tamaño
        if self.index_type == "auto":
            if self.n_vectors < 10_000:
                self.index_type = "flat"
            elif self.n_vectors < 1_000_000:
                self.index_type = "ivf"
            else:
                self.index_type = "ivfpq"
        
        # Construir índice según tipo
        if self.index_type == "flat":
            # Búsqueda exacta (baseline)
            if self.metric == "cosine":
                self.index = faiss.IndexFlatIP(self.dimension)  # Inner Product
            else:
                self.index = faiss.IndexFlatL2(self.dimension)
            self.index.add(embeddings)
            print(f"✓ IndexFlat construido: {self.n_vectors} vectores")
        
        elif self.index_type == "ivf":
            # IVF: particionado con K-Means
            nlist = nlist or min(int(np.sqrt(self.n_vectors)), 4096)
            
            if self.metric == "cosine":
                quantizer = faiss.IndexFlatIP(self.dimension)
                self.index = faiss.IndexIVFFlat(quantizer, self.dimension, nlist)
            else:
                quantizer = faiss.IndexFlatL2(self.dimension)
                self.index = faiss.IndexIVFFlat(quantizer, self.dimension, nlist)
            
            # Training: K-Means para encontrar centroides
            print(f"Training IVF con {nlist} clusters...")
            self.index.train(embeddings)
            self.index.add(embeddings)
            
            # Configurar nprobe (cuántos clusters buscar)
            self.index.nprobe = min(10, nlist // 10)  # Default: 10 o 10% de clusters
            print(f"✓ IndexIVFFlat construido: {self.n_vectors} vectores, {nlist} clusters, nprobe={self.index.nprobe}")
        
        elif self.index_type == "ivfpq":
            # IVFPQ: particionado + compresión
            nlist = nlist or min(int(np.sqrt(self.n_vectors)), 4096)
            m = 8  # Número de subvectores (debe dividir dimension)
            nbits = 8  # Bits por subvector (2^8 = 256 centroides por subvector)
            
            if self.metric == "cosine":
                quantizer = faiss.IndexFlatIP(self.dimension)
                self.index = faiss.IndexIVFPQ(quantizer, self.dimension, nlist, m, nbits)
            else:
                quantizer = faiss.IndexFlatL2(self.dimension)
                self.index = faiss.IndexIVFPQ(quantizer, self.dimension, nlist, m, nbits)
            
            print(f"Training IVFPQ con {nlist} clusters, m={m}, nbits={nbits}...")
            self.index.train(embeddings)
            self.index.add(embeddings)
            
            self.index.nprobe = min(10, nlist // 10)
            
            # Calcular compresión
            original_size = self.n_vectors * self.dimension * 4  # float32
            compressed_size = self.n_vectors * m  # m bytes por vector
            ratio = original_size / compressed_size
            print(f"✓ IndexIVFPQ construido: {self.n_vectors} vectores, compresión {ratio:.1f}x")
        
        elif self.index_type == "hnsw":
            # HNSW: grafo jerárquico
            M = 32  # Número de conexiones por nodo
            self.index = faiss.IndexHNSWFlat(self.dimension, M)
            self.index.hnsw.efConstruction = 40  # Calidad de construcción
            self.index.add(embeddings)
            print(f"✓ IndexHNSW construido: {self.n_vectors} vectores, M={M}")
        
        else:
            raise ValueError(f"Tipo de índice no soportado: {self.index_type}")
    
    def search(
        self, 
        queries: np.ndarray, 
        k: int = 10
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Busca k vecinos más cercanos para cada query.
        
        Args:
            queries: (n_queries, dim) matriz de queries
            k: número de vecinos a retornar
        
        Returns:
            distances: (n_queries, k) distancias/similitudes
            indices: (n_queries, k) índices de vecinos
        """
        queries = queries.astype('float32')
        
        if self.metric == "cosine":
            faiss.normalize_L2(queries)
        
        distances, indices = self.index.search(queries, k)
        return distances, indices
    
    def tune_parameters(self, queries: np.ndarray, ground_truth_indices: np.ndarray):
        """
        Encuentra nprobe óptimo balanceando recall vs latencia.
        
        Args:
            queries: queries de validación
            ground_truth_indices: resultados exactos (de IndexFlat)
        """
        if self.index_type not in ["ivf", "ivfpq"]:
            print("Tuning solo aplica a índices IVF")
            return
        
        nprobe_values = [1, 2, 5, 10, 20, 50, 100]
        results = []
        
        for nprobe in nprobe_values:
            self.index.nprobe = nprobe
            
            start = time.perf_counter()
            _, indices = self.search(queries, k=10)
            latency = (time.perf_counter() - start) * 1000 / len(queries)
            
            # Calcular recall@10
            recall = np.mean([
                len(np.intersect1d(indices[i], ground_truth_indices[i])) / 10
                for i in range(len(queries))
            ])
            
            results.append({
                "nprobe": nprobe,
                "recall": recall,
                "latency_ms": latency
            })
            
            print(f"nprobe={nprobe:3} | recall@10={recall:.3f} | latency={latency:.2f}ms")
        
        # Seleccionar nprobe con recall >0.95 y menor latencia
        best = max([r for r in results if r["recall"] >= 0.95], 
                   key=lambda x: -x["latency_ms"], 
                   default=results[-1])
        
        self.index.nprobe = best["nprobe"]
        print(f"\n✓ Configuración óptima: nprobe={best['nprobe']} (recall={best['recall']:.3f}, latency={best['latency_ms']:.2f}ms)")
    
    def save(self, filepath: str):
        """Guarda índice a disco."""
        faiss.write_index(self.index, filepath)
        print(f"✓ Índice guardado en {filepath}")
    
    def load(self, filepath: str):
        """Carga índice desde disco."""
        self.index = faiss.read_index(filepath)
        self.n_vectors = self.index.ntotal
        print(f"✓ Índice cargado desde {filepath}: {self.n_vectors} vectores")

# Ejemplo: construcción y búsqueda con FAISS
np.random.seed(42)
n_vectors = 100_000
dimension = 768

print("Generando embeddings...")
embeddings = np.random.randn(n_vectors, dimension).astype('float32')

# Construir índice IVF
engine = FAISSSearchEngine(dimension=dimension, index_type="ivf", metric="cosine")
engine.build_index(embeddings)

# Búsqueda
queries = np.random.randn(100, dimension).astype('float32')
k = 10

start = time.perf_counter()
distances, indices = engine.search(queries, k=k)
elapsed = (time.perf_counter() - start) * 1000

print(f"\nBúsqueda completada:")
print(f"  {len(queries)} queries × top-{k}")
print(f"  Latencia: {elapsed/len(queries):.2f} ms/query")
print(f"  Throughput: {len(queries)/(elapsed/1000):.0f} queries/seg")

# Guardar índice
engine.save("faiss_index.bin")
```

### 📊 Comparación FAISS vs Annoy vs Brute Force

```python
import annoy

def benchmark_search_engines(embeddings: np.ndarray, queries: np.ndarray, k: int = 10):
    """Compara diferentes motores de búsqueda."""
    n, dim = embeddings.shape
    
    # Normalizar para cosine similarity
    embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    queries_norm = queries / np.linalg.norm(queries, axis=1, keepdims=True)
    
    results = {}
    
    # 1. Brute Force (NumPy)
    print("1️⃣ Brute Force (NumPy)...")
    start = time.perf_counter()
    scores = queries_norm @ embeddings_norm.T  # (n_queries, n_corpus)
    bf_indices = np.argsort(scores, axis=1)[:, -k:][:, ::-1]
    bf_time = (time.perf_counter() - start) * 1000
    
    results["brute_force"] = {
        "build_time_ms": 0,
        "search_time_ms": bf_time,
        "latency_per_query_ms": bf_time / len(queries),
        "recall": 1.0,  # 100% por definición
        "memory_mb": embeddings.nbytes / 1024 / 1024
    }
    
    # 2. FAISS IndexFlatIP (exacto optimizado)
    print("2️⃣ FAISS IndexFlatIP...")
    start = time.perf_counter()
    index_flat = faiss.IndexFlatIP(dim)
    index_flat.add(embeddings_norm.astype('float32'))
    build_time = (time.perf_counter() - start) * 1000
    
    start = time.perf_counter()
    faiss_flat_distances, faiss_flat_indices = index_flat.search(queries_norm.astype('float32'), k)
    search_time = (time.perf_counter() - start) * 1000
    
    results["faiss_flat"] = {
        "build_time_ms": build_time,
        "search_time_ms": search_time,
        "latency_per_query_ms": search_time / len(queries),
        "recall": 1.0,
        "memory_mb": embeddings.nbytes / 1024 / 1024
    }
    
    # 3. FAISS IndexIVFFlat (aproximado)
    print("3️⃣ FAISS IndexIVFFlat...")
    nlist = min(int(np.sqrt(n)), 1000)
    quantizer = faiss.IndexFlatIP(dim)
    index_ivf = faiss.IndexIVFFlat(quantizer, dim, nlist)
    
    start = time.perf_counter()
    index_ivf.train(embeddings_norm.astype('float32'))
    index_ivf.add(embeddings_norm.astype('float32'))
    build_time = (time.perf_counter() - start) * 1000
    
    index_ivf.nprobe = 10
    start = time.perf_counter()
    faiss_ivf_distances, faiss_ivf_indices = index_ivf.search(queries_norm.astype('float32'), k)
    search_time = (time.perf_counter() - start) * 1000
    
    # Calcular recall vs brute force
    recall = np.mean([
        len(np.intersect1d(faiss_ivf_indices[i], bf_indices[i])) / k
        for i in range(len(queries))
    ])
    
    results["faiss_ivf"] = {
        "build_time_ms": build_time,
        "search_time_ms": search_time,
        "latency_per_query_ms": search_time / len(queries),
        "recall": recall,
        "memory_mb": embeddings.nbytes / 1024 / 1024
    }
    
    # 4. Annoy (Spotify)
    print("4️⃣ Annoy...")
    annoy_index = annoy.AnnoyIndex(dim, 'angular')  # angular = cosine
    
    start = time.perf_counter()
    for i, vec in enumerate(embeddings_norm):
        annoy_index.add_item(i, vec)
    annoy_index.build(10)  # 10 árboles (más árboles = mejor recall pero más lento)
    build_time = (time.perf_counter() - start) * 1000
    
    start = time.perf_counter()
    annoy_indices = []
    for query in queries_norm:
        annoy_indices.append(annoy_index.get_nns_by_vector(query, k))
    search_time = (time.perf_counter() - start) * 1000
    
    annoy_indices = np.array(annoy_indices)
    recall = np.mean([
        len(np.intersect1d(annoy_indices[i], bf_indices[i])) / k
        for i in range(len(queries))
    ])
    
    # Annoy guarda índice en disco, estimar tamaño
    annoy_index.save("temp_annoy.ann")
    import os
    memory_mb = os.path.getsize("temp_annoy.ann") / 1024 / 1024
    os.remove("temp_annoy.ann")
    
    results["annoy"] = {
        "build_time_ms": build_time,
        "search_time_ms": search_time,
        "latency_per_query_ms": search_time / len(queries),
        "recall": recall,
        "memory_mb": memory_mb
    }
    
    return results

# Ejecutar benchmark
embeddings = np.random.randn(50000, 384).astype('float32')
queries = np.random.randn(100, 384).astype('float32')

bench_results = benchmark_search_engines(embeddings, queries, k=10)

# Mostrar resultados
print("\n" + "="*80)
print(f"{'Engine':<15} | {'Build (ms)':<12} | {'Search (ms)':<12} | {'Latency/Q (ms)':<15} | {'Recall':<8} | {'Memory (MB)':<12}")
print("="*80)
for engine, metrics in bench_results.items():
    print(f"{engine:<15} | {metrics['build_time_ms']:>11.2f} | {metrics['search_time_ms']:>11.2f} | {metrics['latency_per_query_ms']:>14.3f} | {metrics['recall']:>7.3f} | {metrics['memory_mb']:>11.1f}")

# Salida esperada:
# ================================================================================
# Engine          | Build (ms)   | Search (ms)  | Latency/Q (ms) | Recall   | Memory (MB) 
# ================================================================================
# brute_force     |        0.00 |      342.56 |         3.426 |   1.000 |        73.2
# faiss_flat      |       12.34 |       89.12 |         0.891 |   1.000 |        73.2
# faiss_ivf       |      234.56 |       15.67 |         0.157 |   0.943 |        73.2
# annoy           |     1234.89 |       12.34 |         0.123 |   0.921 |        95.6
```

### 🎯 Guía de Selección de Índice

| Escala | Índice Recomendado | Build Time | Search Latency | Recall | Memory | Caso de Uso |
|--------|-------------------|------------|----------------|--------|--------|-------------|
| <10K | **IndexFlat** | Instantáneo | 1-5 ms/query | 100% | 1x | Desarrollo, validación |
| 10K-100K | **IndexHNSW** | Medio | 0.5-2 ms/query | 95-99% | 1x | Producción, alta precisión |
| 100K-1M | **IndexIVFFlat** | Medio | 0.1-1 ms/query | 90-95% | 1x | Producción general |
| 1M-10M | **IndexIVFPQ** | Alto | 0.05-0.5 ms/query | 85-90% | 0.1x | Alta escala, cost-effective |
| >10M | **IndexIVFPQ + GPU** | Muy alto | 0.01-0.1 ms/query | 85-90% | 0.1x | Web-scale search |

### 💡 Optimizaciones Avanzadas

```python
# 1. GPU Acceleration (100x speedup para corpus muy grande)
import faiss
gpu_resources = faiss.StandardGpuResources()
index_cpu = faiss.IndexIVFFlat(quantizer, dimension, nlist)
index_gpu = faiss.index_cpu_to_gpu(gpu_resources, 0, index_cpu)  # 0 = GPU ID

# 2. Sharding para corpus masivo (>100M vectores)
# Dividir corpus en shards, buscar en paralelo, merge results
shards = [
    FAISSSearchEngine(dim, "ivfpq") for _ in range(10)
]
for i, shard in enumerate(shards):
    shard.build_index(embeddings[i*chunk_size:(i+1)*chunk_size])

# Búsqueda paralela
from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(lambda s: s.search(query, k=100), shards))
# Merge top-100 de cada shard → global top-10

# 3. Prefiltering con metadatos
# FAISS no soporta filtros nativamente, usar ID selector
ids_filtered = [i for i, meta in enumerate(metadata) if meta['type'] == 'pipeline']
selector = faiss.IDSelectorArray(len(ids_filtered), faiss.swig_ptr(np.array(ids_filtered)))
distances, indices = index.search(query, k=10, params=faiss.SearchParametersIVF(sel=selector))
```

### 🚀 Reglas de Oro para Búsqueda Vectorial a Escala

1. **<10K vectores**: Usar búsqueda lineal (NumPy) - es suficiente
2. **10K-100K**: FAISS **IndexHNSW** (mejor recall, no requiere training)
3. **>100K**: FAISS **IndexIVFFlat** con nprobe tuneado (balance recall/latencia)
4. **>1M**: FAISS **IndexIVFPQ** para reducir memoria 10x
5. **Siempre** medir recall vs ground truth antes de producción (target: >95%)
6. **Tune nprobe** con queries reales, no valores por defecto
7. **Guardar índice** entrenado a disco, no reconstruir cada deploy
8. **GPU** solo si corpus >10M y latencia <1ms requerida

## 🏭 Aplicaciones en Data Engineering: Deduplicación, Clustering y Recomendaciones

Los embeddings permiten resolver problemas complejos de Data Engineering que antes requerían reglas manuales o lógica difícil de mantener: **deduplicación fuzzy**, **clustering semántico**, **recomendaciones** de datasets, y **etiquetado automático** de activos de datos.

### 🔍 1. Deduplicación Inteligente de Registros

La deduplicación tradicional (exact match por hash) falla con variaciones: "Apple iPhone 13" vs "iPhone 13 by Apple". Los embeddings capturan similitud semántica.

```
┌─────────────────────────────────────────────────────────────────┐
│              PIPELINE DE DEDUPLICACIÓN CON EMBEDDINGS            │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  Input: Tabla con posibles duplicados                           │
│  ┌────────────────────────────────────────────────┐             │
│  │ id │ descripcion                               │             │
│  ├────┼───────────────────────────────────────────┤             │
│  │ 1  │ Apple iPhone 13 Pro Max 256GB             │             │
│  │ 2  │ iPhone 13 Pro Max 256GB Apple             │ ← Duplicado │
│  │ 3  │ Samsung Galaxy S21 Ultra                  │             │
│  │ 4  │ Galaxy S21 Ultra by Samsung               │ ← Duplicado │
│  │ 5  │ Sony PlayStation 5 Console                │             │
│  └────────────────────────────────────────────────┘             │
│        ↓                                                         │
│  1️⃣ Preprocesamiento                                            │
│     - Lowercasing: "Apple iPhone" → "apple iphone"             │
│     - Remover stopwords: "by", "the", etc.                     │
│     - Normalizar espacios: "  " → " "                          │
│     - Quitar caracteres especiales: "iPhone-13" → "iphone 13"  │
│        ↓                                                         │
│  2️⃣ Generación de Embeddings                                    │
│     - Usar modelo robusto (bge-large-en-v1.5)                  │
│     - Batch processing (100 registros/llamada)                 │
│     - Cache embeddings en columna nueva                        │
│        ↓                                                         │
│  3️⃣ Búsqueda de Pares Similares                                │
│     - Opción A: All-pairs O(n²) para corpus pequeño           │
│     - Opción B: FAISS para corpus grande (>100K)              │
│     - Threshold típico: 0.95-0.99 (percentil 99 del corpus)   │
│        ↓                                                         │
│  4️⃣ Clustering de Duplicados                                    │
│     - Connected components: transitivity                       │
│       Si A≈B y B≈C → {A, B, C} es cluster                     │
│     - Asignar cluster_id a cada registro                       │
│        ↓                                                         │
│  5️⃣ Resolución (elegir registro canónico)                       │
│     - Estrategia 1: El más completo (más palabras)            │
│     - Estrategia 2: El más reciente (timestamp)               │
│     - Estrategia 3: El más popular (más ventas/views)         │
│        ↓                                                         │
│  Output: Tabla deduplicada + mapping                            │
│  ┌────────────────────────────────────────────────┐             │
│  │ cluster_id │ canonical_id │ descripcion        │             │
│  ├────────────┼──────────────┼────────────────────┤             │
│  │ 1          │ 1            │ Apple iPhone 13... │             │
│  │ 1          │ 1            │ iPhone 13 Pro Max..│ → merged   │
│  │ 2          │ 3            │ Samsung Galaxy...  │             │
│  │ 2          │ 3            │ Galaxy S21 Ultra...│ → merged   │
│  │ 3          │ 5            │ Sony PlayStation...│             │
│  └────────────────────────────────────────────────┘             │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Implementación Completa

```python
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from dataclasses import dataclass

@dataclass
class DuplicateCluster:
    """Cluster de registros duplicados."""
    cluster_id: int
    record_ids: List[int]
    canonical_id: int
    similarity_scores: List[float]

class FuzzyDeduplicator:
    """Deduplicador fuzzy usando embeddings."""
    
    def __init__(self, embedding_generator, threshold: float = 0.95):
        """
        Args:
            embedding_generator: EmbeddingGenerator instance
            threshold: similitud mínima para considerar duplicados (0.95-0.99)
        """
        self.generator = embedding_generator
        self.threshold = threshold
    
    def preprocess_text(self, text: str) -> str:
        """Normaliza texto antes de embeddings."""
        import re
        text = text.lower()
        text = re.sub(r'[^\w\s]', ' ', text)  # Remover puntuación
        text = re.sub(r'\s+', ' ', text).strip()  # Normalizar espacios
        return text
    
    def find_duplicates(
        self, 
        df: pd.DataFrame, 
        text_column: str,
        id_column: str = None
    ) -> pd.DataFrame:
        """
        Encuentra duplicados en DataFrame.
        
        Args:
            df: DataFrame con registros
            text_column: columna con texto a comparar
            id_column: columna con ID único (usa index si None)
        
        Returns:
            DataFrame con cluster_id y canonical_id
        """
        if id_column is None:
            df = df.reset_index(drop=True)
            id_column = 'index'
            df[id_column] = df.index
        
        # 1. Preprocesamiento
        print("1️⃣ Preprocesando texto...")
        df['text_clean'] = df[text_column].apply(self.preprocess_text)
        
        # 2. Generar embeddings (con cache)
        print("2️⃣ Generando embeddings...")
        embeddings = self.generator.embed(df['text_clean'].tolist())
        
        # 3. Encontrar pares similares
        print(f"3️⃣ Buscando pares con similitud >{self.threshold}...")
        pairs = self._find_similar_pairs(embeddings, df[id_column].tolist())
        print(f"   Encontrados {len(pairs)} pares de duplicados")
        
        # 4. Clustering (connected components)
        print("4️⃣ Agrupando en clusters...")
        clusters = self._cluster_duplicates(pairs, df[id_column].tolist())
        print(f"   {len(clusters)} clusters de duplicados")
        
        # 5. Seleccionar registros canónicos
        print("5️⃣ Seleccionando registros canónicos...")
        cluster_map = {}
        for cluster in clusters:
            canonical_id = self._select_canonical(cluster, df, text_column, id_column)
            for record_id in cluster.record_ids:
                cluster_map[record_id] = {
                    'cluster_id': cluster.cluster_id,
                    'canonical_id': canonical_id,
                    'is_canonical': record_id == canonical_id
                }
        
        # Agregar columnas al DataFrame
        df['cluster_id'] = df[id_column].map(lambda x: cluster_map.get(x, {}).get('cluster_id'))
        df['canonical_id'] = df[id_column].map(lambda x: cluster_map.get(x, {}).get('canonical_id', x))
        df['is_duplicate'] = df['cluster_id'].notna()
        df['is_canonical'] = df[id_column].map(lambda x: cluster_map.get(x, {}).get('is_canonical', True))
        
        return df
    
    def _find_similar_pairs(
        self, 
        embeddings: np.ndarray, 
        ids: List
    ) -> List[Tuple[int, int, float]]:
        """Encuentra pares de embeddings similares."""
        pairs = []
        
        # Para corpus pequeño (<10K), usar all-pairs
        if len(embeddings) < 10_000:
            sim_matrix = cosine_similarity(embeddings)
            for i in range(len(embeddings)):
                for j in range(i+1, len(embeddings)):
                    if sim_matrix[i][j] > self.threshold:
                        pairs.append((ids[i], ids[j], sim_matrix[i][j]))
        else:
            # Para corpus grande, usar FAISS
            import faiss
            embeddings_norm = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
            
            index = faiss.IndexFlatIP(embeddings.shape[1])
            index.add(embeddings_norm.astype('float32'))
            
            # Buscar k vecinos más cercanos para cada vector
            k = min(10, len(embeddings) - 1)
            distances, indices = index.search(embeddings_norm.astype('float32'), k + 1)
            
            for i in range(len(embeddings)):
                for j, dist in zip(indices[i][1:], distances[i][1:]):  # Excluir el mismo vector
                    if dist > self.threshold and ids[i] < ids[j]:  # Evitar duplicar pares
                        pairs.append((ids[i], int(j), float(dist)))
        
        return pairs
    
    def _cluster_duplicates(
        self, 
        pairs: List[Tuple[int, int, float]], 
        all_ids: List[int]
    ) -> List[DuplicateCluster]:
        """Agrupa pares en clusters usando connected components."""
        # Construir grafo
        G = nx.Graph()
        G.add_nodes_from(all_ids)
        
        for id1, id2, score in pairs:
            G.add_edge(id1, id2, weight=score)
        
        # Encontrar componentes conexas
        clusters = []
        for i, component in enumerate(nx.connected_components(G)):
            if len(component) > 1:  # Solo clusters con >1 registro
                component_list = list(component)
                
                # Calcular similitudes promedio
                scores = [
                    G[u][v]['weight'] 
                    for u, v in G.edges(component_list)
                ]
                
                clusters.append(DuplicateCluster(
                    cluster_id=i,
                    record_ids=component_list,
                    canonical_id=component_list[0],  # Temporal, se reemplaza después
                    similarity_scores=scores
                ))
        
        return clusters
    
    def _select_canonical(
        self, 
        cluster: DuplicateCluster, 
        df: pd.DataFrame, 
        text_column: str,
        id_column: str
    ) -> int:
        """Selecciona registro canónico del cluster."""
        cluster_df = df[df[id_column].isin(cluster.record_ids)]
        
        # Estrategia: el registro más completo (más palabras)
        cluster_df['word_count'] = cluster_df[text_column].str.split().str.len()
        canonical_id = cluster_df.loc[cluster_df['word_count'].idxmax(), id_column]
        
        return canonical_id
    
    def get_deduplication_report(self, df: pd.DataFrame) -> Dict:
        """Genera reporte de deduplicación."""
        total_records = len(df)
        duplicate_records = df['is_duplicate'].sum()
        unique_records = total_records - duplicate_records
        n_clusters = df['cluster_id'].nunique()
        
        # Distribución de tamaño de clusters
        cluster_sizes = df[df['is_duplicate']].groupby('cluster_id').size()
        
        return {
            "total_records": total_records,
            "unique_records": unique_records,
            "duplicate_records": duplicate_records,
            "deduplication_rate": duplicate_records / total_records,
            "n_clusters": n_clusters,
            "avg_cluster_size": cluster_sizes.mean(),
            "max_cluster_size": cluster_sizes.max(),
            "cluster_size_distribution": cluster_sizes.value_counts().to_dict()
        }

# Ejemplo de uso
from sentence_transformers import SentenceTransformer

# Dataset de ejemplo
data = pd.DataFrame({
    'id': range(1, 11),
    'description': [
        'Apple iPhone 13 Pro Max 256GB',
        'iPhone 13 Pro Max 256GB Apple',
        'Samsung Galaxy S21 Ultra',
        'Galaxy S21 Ultra by Samsung',
        'Sony PlayStation 5 Console',
        'PS5 Console by Sony',
        'MacBook Pro 14 inch M1',
        'Apple MacBook Pro 14" M1 Chip',
        'Nike Air Max Sneakers',
        'Dell XPS 13 Laptop'
    ]
})

# Deduplicación
generator = EmbeddingGenerator("bge")  # Usar modelo open source
deduplicator = FuzzyDeduplicator(generator, threshold=0.90)

df_dedup = deduplicator.find_duplicates(data, text_column='description', id_column='id')

# Mostrar resultados
print("\nRegistros con duplicados:")
print(df_dedup[df_dedup['is_duplicate']][['id', 'description', 'cluster_id', 'canonical_id', 'is_canonical']])

# Reporte
report = deduplicator.get_deduplication_report(df_dedup)
print(f"\n📊 Reporte de Deduplicación:")
print(f"  Total registros: {report['total_records']}")
print(f"  Únicos: {report['unique_records']}")
print(f"  Duplicados: {report['duplicate_records']} ({report['deduplication_rate']:.1%})")
print(f"  Clusters: {report['n_clusters']}")
print(f"  Tamaño promedio de cluster: {report['avg_cluster_size']:.1f}")
```

### 🎯 2. Clustering Semántico de Datasets

Agrupar tablas/pipelines por dominio de negocio sin etiquetas manuales.

```python
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

class DatasetClusterer:
    """Agrupa datasets por similitud semántica."""
    
    def __init__(self, embedding_generator):
        self.generator = embedding_generator
    
    def cluster_kmeans(
        self, 
        descriptions: List[str], 
        n_clusters: int = None,
        auto_k: bool = True
    ) -> Tuple[np.ndarray, Dict]:
        """
        Clustering con K-Means.
        
        Args:
            descriptions: lista de descripciones de datasets
            n_clusters: número de clusters (auto-detecta si None)
            auto_k: usar elbow method para encontrar k óptimo
        """
        # Generar embeddings
        embeddings = self.generator.embed(descriptions)
        
        # Auto-detectar k óptimo
        if auto_k and n_clusters is None:
            n_clusters = self._find_optimal_k(embeddings)
            print(f"✓ K óptimo detectado: {n_clusters}")
        elif n_clusters is None:
            n_clusters = 5  # Default
        
        # K-Means
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(embeddings)
        
        # Calcular métricas
        from sklearn.metrics import silhouette_score, calinski_harabasz_score
        metrics = {
            "n_clusters": n_clusters,
            "silhouette_score": silhouette_score(embeddings, labels),  # [-1, 1], >0.5 bueno
            "calinski_harabasz": calinski_harabasz_score(embeddings, labels),  # >1000 bueno
            "cluster_sizes": np.bincount(labels).tolist()
        }
        
        # Encontrar términos representativos por cluster
        cluster_terms = self._extract_cluster_terms(descriptions, labels, n_clusters)
        metrics["cluster_terms"] = cluster_terms
        
        return labels, metrics
    
    def cluster_dbscan(
        self, 
        descriptions: List[str], 
        eps: float = 0.3,
        min_samples: int = 2
    ) -> Tuple[np.ndarray, Dict]:
        """
        Clustering con DBSCAN (density-based, no requiere k).
        
        Args:
            descriptions: lista de descripciones
            eps: distancia máxima para considerar vecinos (0.1-0.5)
            min_samples: mínimo de vecinos para formar cluster
        """
        embeddings = self.generator.embed(descriptions)
        
        # DBSCAN con distancia coseno
        from sklearn.metrics.pairwise import cosine_distances
        distance_matrix = cosine_distances(embeddings)
        
        dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='precomputed')
        labels = dbscan.fit_predict(distance_matrix)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        metrics = {
            "n_clusters": n_clusters,
            "n_noise": n_noise,  # Puntos no asignados a ningún cluster
            "cluster_sizes": {
                int(label): int(count) 
                for label, count in zip(*np.unique(labels, return_counts=True))
            }
        }
        
        return labels, metrics
    
    def _find_optimal_k(self, embeddings: np.ndarray, k_range: range = range(2, 11)) -> int:
        """Encuentra k óptimo usando elbow method."""
        inertias = []
        for k in k_range:
            kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
            kmeans.fit(embeddings)
            inertias.append(kmeans.inertia_)
        
        # Encontrar "codo" (mayor cambio en pendiente)
        diffs = np.diff(inertias)
        diff_ratios = diffs[:-1] / diffs[1:]
        optimal_k = k_range[np.argmax(diff_ratios) + 1]
        
        return optimal_k
    
    def _extract_cluster_terms(
        self, 
        descriptions: List[str], 
        labels: np.ndarray, 
        n_clusters: int
    ) -> Dict[int, List[str]]:
        """Extrae términos más frecuentes por cluster."""
        from collections import Counter
        import re
        
        cluster_terms = {}
        for cluster_id in range(n_clusters):
            # Obtener descripciones del cluster
            cluster_docs = [
                desc for desc, label in zip(descriptions, labels) 
                if label == cluster_id
            ]
            
            # Tokenizar y contar términos
            all_words = []
            for doc in cluster_docs:
                words = re.findall(r'\b\w+\b', doc.lower())
                all_words.extend([w for w in words if len(w) > 3])  # Filtrar palabras cortas
            
            # Top-5 términos
            counter = Counter(all_words)
            cluster_terms[cluster_id] = [word for word, count in counter.most_common(5)]
        
        return cluster_terms
    
    def visualize_clusters(
        self, 
        embeddings: np.ndarray, 
        labels: np.ndarray, 
        descriptions: List[str]
    ):
        """Visualiza clusters en 2D con PCA."""
        # Reducir a 2D
        pca = PCA(n_components=2, random_state=42)
        coords_2d = pca.fit_transform(embeddings)
        
        # Plot
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(
            coords_2d[:, 0], 
            coords_2d[:, 1], 
            c=labels, 
            cmap='tab10', 
            alpha=0.6,
            s=100
        )
        
        # Anotar algunos puntos
        for i in range(min(15, len(descriptions))):
            plt.annotate(
                descriptions[i][:40], 
                (coords_2d[i, 0], coords_2d[i, 1]),
                fontsize=8,
                alpha=0.7
            )
        
        plt.colorbar(scatter, label='Cluster ID')
        plt.title('Clustering de Datasets por Similitud Semántica')
        plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} varianza)')
        plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} varianza)')
        plt.tight_layout()
        plt.show()

# Ejemplo: clustering de catálogo de datos
catalog_descriptions = [
    "Transacciones de venta diarias con monto y producto",
    "Data de ventas por día con ingresos totales",
    "Información demográfica de clientes: edad, ciudad, segmento",
    "Datos de clientes con perfil demográfico completo",
    "Inventario de productos por almacén con stock disponible",
    "Stock de productos en todos los almacenes",
    "Métricas de engagement de usuarios en plataforma",
    "Actividad de usuarios: clicks, sesiones, tiempo en sitio",
    "Campañas de marketing con presupuesto y ROI",
    "Data de campañas publicitarias y performance"
]

generator = EmbeddingGenerator("sbert")
clusterer = DatasetClusterer(generator)

labels, metrics = clusterer.cluster_kmeans(catalog_descriptions, auto_k=True)

print(f"📊 Resultados de Clustering:")
print(f"  Clusters: {metrics['n_clusters']}")
print(f"  Silhouette Score: {metrics['silhouette_score']:.3f} (>0.5 es bueno)")
print(f"  Tamaños: {metrics['cluster_sizes']}")
print(f"\n  Términos por cluster:")
for cluster_id, terms in metrics['cluster_terms'].items():
    print(f"    Cluster {cluster_id}: {', '.join(terms)}")
```

### 🎁 3. Sistema de Recomendación de Datasets

```python
class DatasetRecommender:
    """Recomienda datasets similares para analistas."""
    
    def __init__(self, embedding_generator):
        self.generator = embedding_generator
        self.catalog_embeddings = None
        self.catalog_metadata = None
    
    def index_catalog(self, catalog: pd.DataFrame, text_column: str):
        """Indexa catálogo de datasets."""
        descriptions = catalog[text_column].tolist()
        self.catalog_embeddings = self.generator.embed(descriptions)
        self.catalog_metadata = catalog.to_dict('records')
        print(f"✓ Catálogo indexado: {len(catalog)} datasets")
    
    def recommend_similar(
        self, 
        query: str, 
        top_k: int = 5,
        filters: Dict = None
    ) -> List[Dict]:
        """
        Recomienda datasets similares a la query.
        
        Args:
            query: descripción de lo que busca el usuario
            top_k: número de recomendaciones
            filters: filtros de metadatos {"owner": "analytics", "type": "table"}
        """
        # Embedding de query
        query_emb = self.generator.embed_single(query)
        
        # Aplicar filtros de metadatos
        if filters:
            mask = np.ones(len(self.catalog_metadata), dtype=bool)
            for key, value in filters.items():
                mask &= np.array([meta.get(key) == value for meta in self.catalog_metadata])
            
            filtered_embeddings = self.catalog_embeddings[mask]
            filtered_metadata = [m for m, include in zip(self.catalog_metadata, mask) if include]
        else:
            filtered_embeddings = self.catalog_embeddings
            filtered_metadata = self.catalog_metadata
        
        # Calcular similitudes
        similarities = cosine_similarity([query_emb], filtered_embeddings)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        # Construir resultados
        recommendations = []
        for idx in top_indices:
            rec = filtered_metadata[idx].copy()
            rec['similarity'] = float(similarities[idx])
            rec['relevance'] = "Alta" if similarities[idx] > 0.8 else "Media" if similarities[idx] > 0.6 else "Baja"
            recommendations.append(rec)
        
        return recommendations
    
    def recommend_collaborative(
        self, 
        user_history: List[str], 
        top_k: int = 5
    ) -> List[Dict]:
        """
        Recomendaciones colaborativas basadas en historial del usuario.
        
        Args:
            user_history: lista de dataset_ids que el usuario ha usado
        """
        # Embeddings de datasets usados
        used_indices = [
            i for i, meta in enumerate(self.catalog_metadata)
            if meta.get('dataset_id') in user_history
        ]
        used_embeddings = self.catalog_embeddings[used_indices]
        
        # Centroid del perfil del usuario
        user_profile = used_embeddings.mean(axis=0)
        
        # Recomendar datasets similares al perfil (excluyendo ya usados)
        available_mask = np.array([
            meta.get('dataset_id') not in user_history 
            for meta in self.catalog_metadata
        ])
        
        available_embeddings = self.catalog_embeddings[available_mask]
        available_metadata = [m for m, avail in zip(self.catalog_metadata, available_mask) if avail]
        
        similarities = cosine_similarity([user_profile], available_embeddings)[0]
        top_indices = np.argsort(similarities)[-top_k:][::-1]
        
        recommendations = []
        for idx in top_indices:
            rec = available_metadata[idx].copy()
            rec['similarity'] = float(similarities[idx])
            recommendations.append(rec)
        
        return recommendations

# Ejemplo: recomendaciones
catalog = pd.DataFrame({
    'dataset_id': ['dwh.ventas', 'dwh.clientes', 'dwh.productos', 'analytics.revenue', 'analytics.engagement'],
    'description': [
        'Transacciones de venta con fecha, monto, producto',
        'Datos demográficos de clientes: edad, ciudad, segmento',
        'Catálogo de productos con precios y categorías',
        'Ingresos mensuales agregados por región',
        'Métricas de engagement de usuarios en plataforma'
    ],
    'owner': ['data-eng', 'data-eng', 'data-eng', 'analytics', 'analytics'],
    'type': ['table', 'table', 'table', 'view', 'view']
})

recommender = DatasetRecommender(generator)
recommender.index_catalog(catalog, text_column='description')

# Búsqueda semántica
print("🔍 Buscar: 'necesito datos de ventas mensuales'")
results = recommender.recommend_similar('necesito datos de ventas mensuales', top_k=3)
for i, rec in enumerate(results, 1):
    print(f"{i}. {rec['dataset_id']} (sim={rec['similarity']:.3f}, {rec['relevance']})")
    print(f"   {rec['description']}")

# Recomendaciones colaborativas
print("\n🎁 Recomendaciones basadas en historial: ['dwh.ventas']")
collab_recs = recommender.recommend_collaborative(['dwh.ventas'], top_k=3)
for i, rec in enumerate(collab_recs, 1):
    print(f"{i}. {rec['dataset_id']} (sim={rec['similarity']:.3f})")
```

### 📊 4. Detección de Anomalías en Metadatos

Identificar tablas "raras" que no encajan con el resto del catálogo.

```python
class AnomalyDetector:
    """Detecta datasets anómalos por similitud semántica."""
    
    def __init__(self, embedding_generator):
        self.generator = embedding_generator
    
    def detect_anomalies(
        self, 
        descriptions: List[str], 
        contamination: float = 0.05
    ) -> Tuple[np.ndarray, np.ndarray]:
        """
        Detecta descripciones anómalas usando Isolation Forest.
        
        Args:
            descriptions: lista de descripciones
            contamination: proporción esperada de anomalías (0.01-0.10)
        
        Returns:
            labels: 1=normal, -1=anomalía
            scores: scores de anomalía (más negativo = más anómalo)
        """
        from sklearn.ensemble import IsolationForest
        
        embeddings = self.generator.embed(descriptions)
        
        iso_forest = IsolationForest(contamination=contamination, random_state=42)
        labels = iso_forest.fit_predict(embeddings)
        scores = iso_forest.score_samples(embeddings)
        
        n_anomalies = (labels == -1).sum()
        print(f"✓ Anomalías detectadas: {n_anomalies}/{len(descriptions)} ({n_anomalies/len(descriptions):.1%})")
        
        return labels, scores

# Ejemplo
descriptions_with_anomaly = catalog_descriptions + [
    "Configuración de base de datos MySQL 5.7",  # Anomalía: no es un dataset
    "Logs de errores de aplicación Python"      # Anomalía: logs técnicos
]

detector = AnomalyDetector(generator)
labels, scores = detector.detect_anomalies(descriptions_with_anomaly, contamination=0.10)

print("\n🚨 Descripciones anómalas:")
for i, (desc, label, score) in enumerate(zip(descriptions_with_anomaly, labels, scores)):
    if label == -1:
        print(f"  [{score:.3f}] {desc}")
```

### 🚀 Métricas de Éxito en Producción

| Aplicación | Métrica | Target | Medición |
|------------|---------|--------|----------|
| **Deduplicación** | Precision | >95% | Manual review de 100 pares |
| **Deduplicación** | Recall | >90% | Cuántos duplicados reales se detectaron |
| **Clustering** | Silhouette Score | >0.5 | sklearn.metrics |
| **Recomendaciones** | Click-through Rate | >20% | % de recomendaciones clickeadas |
| **Recomendaciones** | Relevance@5 | >0.80 | Ratings de usuarios (1-5) |
| **Anomalías** | False Positive Rate | <10% | Manual review de anomalías |

### 💡 Mejores Prácticas en Producción

1. **Deduplicación**: Combinar embeddings + reglas (ej. mismo SKU → 100% duplicado)
2. **Clustering**: Re-entrenar periódicamente al agregar nuevos datasets
3. **Recomendaciones**: Combinar similitud semántica + popularidad + recency
4. **Threshold tuning**: Usar validation set, no adivinar thresholds
5. **Monitoreo**: Loggear similitudes promedio por día (detectar drift)
6. **A/B testing**: Comparar embeddings vs reglas en métricas de negocio

## 1. Generar embeddings

In [ ]:
import os
import numpy as np
from openai import OpenAI

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def embed(text: str) -> np.ndarray:
    resp = client.embeddings.create(
        model='text-embedding-ada-002',
        input=text
    )
    return np.array(resp.data[0].embedding)

textos = [
    'Tabla de ventas con transacciones diarias',
    'Data de transacciones de venta por día',  # Similar
    'Catálogo de productos con precios',
    'Información demográfica de clientes'
]

embeddings = [embed(t) for t in textos]
print(f'Embeddings generados: {len(embeddings)} vectores de dimensión {len(embeddings[0])}')

## 2. Similitud coseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Matriz de similitud
sim_matrix = cosine_similarity(embeddings)

print('Matriz de similitud:\n')
for i, texto in enumerate(textos):
    print(f'{i}. {texto}')

print('\n', sim_matrix.round(3))
print(f'\n➡️ Textos 0 y 1 tienen similitud: {sim_matrix[0][1]:.3f} (duplicados potenciales)')

## 3. Búsqueda semántica en catálogo

In [ ]:
import pandas as pd

# Catálogo de data assets
catalogo = pd.DataFrame([
    {'asset': 'dwh.ventas', 'desc': 'Transacciones de venta con fecha, monto, producto'},
    {'asset': 'dwh.clientes', 'desc': 'Datos demográficos de clientes: edad, ciudad, segmento'},
    {'asset': 'dwh.productos', 'desc': 'Catálogo de productos con precios y categorías'},
    {'asset': 'dwh.inventario', 'desc': 'Stock disponible por almacén y SKU'},
    {'asset': 'analytics.revenue_monthly', 'desc': 'Ingresos mensuales agregados por región'}
])

# Embeddings del catálogo
catalogo['embedding'] = catalogo['desc'].apply(lambda x: embed(x))

def search_catalog(query: str, top_k: int = 3) -> pd.DataFrame:
    query_emb = embed(query)
    catalogo['similarity'] = catalogo['embedding'].apply(
        lambda x: cosine_similarity([query_emb], [x])[0][0]
    )
    return catalogo.nlargest(top_k, 'similarity')[['asset', 'desc', 'similarity']]

print(search_catalog('¿Dónde encuentro información de productos?'))
print('\n')
print(search_catalog('Necesito datos de ventas mensuales'))

## 4. Detección de duplicados

In [ ]:
# Dataset con posibles duplicados
descripciones = [
    'Apple iPhone 13 Pro Max 256GB',
    'iPhone 13 Pro Max 256GB Apple',
    'Samsung Galaxy S21 Ultra',
    'Galaxy S21 Ultra by Samsung',
    'Sony PlayStation 5 Console'
]

embs = [embed(d) for d in descripciones]
threshold = 0.95

duplicates = []
for i in range(len(embs)):
    for j in range(i+1, len(embs)):
        sim = cosine_similarity([embs[i]], [embs[j]])[0][0]
        if sim > threshold:
            duplicates.append((i, j, sim))

print('Duplicados detectados:\n')
for i, j, sim in duplicates:
    print(f'- "{descripciones[i]}" ≈ "{descripciones[j]}" (sim={sim:.3f})')

## 5. Clustering con K-Means

In [ ]:
from sklearn.cluster import KMeans

productos = [
    'Laptop Dell XPS 13',
    'MacBook Pro 14 inch',
    'HP Pavilion Laptop',
    'Nike Air Max Sneakers',
    'Adidas Ultraboost Shoes',
    'Puma Running Shoes',
    'Organic Green Tea 100 bags',
    'Colombian Coffee Beans 1kg'
]

prod_embs = np.array([embed(p) for p in productos])

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(prod_embs)

df_clusters = pd.DataFrame({'producto': productos, 'cluster': clusters})
print(df_clusters.sort_values('cluster'))

## 6. FAISS para búsqueda rápida

In [ ]:
# pip install faiss-cpu
import faiss

# Crear índice
dimension = prod_embs.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(prod_embs.astype('float32'))

# Búsqueda
query = 'zapatos deportivos'
query_emb = embed(query).reshape(1, -1).astype('float32')

k = 3
distances, indices = index.search(query_emb, k)

print(f'Top {k} productos para "{query}":\n')
for i, idx in enumerate(indices[0]):
    print(f'{i+1}. {productos[idx]} (distancia={distances[0][i]:.2f})')

## 7. Recomendaciones

In [ ]:
def recomendar_similar(producto: str, catalogo: list, top_k: int = 3) -> list:
    """Recomienda productos similares."""
    query_emb = embed(producto)
    catalogo_embs = [embed(p) for p in catalogo]
    
    similarities = [
        (p, cosine_similarity([query_emb], [e])[0][0])
        for p, e in zip(catalogo, catalogo_embs)
        if p != producto
    ]
    
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]

comprado = 'Laptop Dell XPS 13'
recomendaciones = recomendar_similar(comprado, productos)

print(f'Cliente compró: {comprado}')
print('Recomendaciones:\n')
for prod, sim in recomendaciones:
    print(f'- {prod} (similitud={sim:.3f})')

## 8. Buenas prácticas

- **Cache embeddings**: generarlos es costoso, almacena en DB.
- **Batch processing**: genera embeddings en lotes para eficiencia.
- **Normalización**: limpia texto antes de generar embeddings.
- **Modelos locales**: considera Sentence Transformers para evitar API calls.
- **Threshold dinámico**: ajusta según caso de uso (duplicados vs búsqueda).
- **Monitoreo de costos**: embeddings consumen tokens.

## 9. Ejercicios

1. Construye un deduplicador de registros usando embeddings.
2. Implementa búsqueda híbrida (keyword + semántica) en tu catálogo.
3. Crea clusters de clientes basados en descripciones de comportamiento.
4. Desarrolla un sistema de recomendación de datasets para analistas.

---

## 🧭 Navegación

**← Anterior:** [📚 RAG: Documentación Técnica con LLMs](04_rag_documentacion_datos.ipynb)

**Siguiente →:** [🤖 Agentes Autónomos para Automatización →](06_agentes_automatizacion.ipynb)

**📚 Índice de Nivel GenAI:**
- [🔄 Comparación: OpenAI vs Google Gemini](00_comparacion_openai_gemini.ipynb)
- [🤖 Fundamentos de LLMs y Prompting](01_fundamentos_llms_prompting.ipynb)
- [📊 Text-to-SQL: Generación de Consultas SQL desde Lenguaje Natural](02_generacion_sql_nl2sql.ipynb)
- [🔧 Generación Automática de Código ETL con LLMs](03_generacion_codigo_etl.ipynb)
- [📚 RAG: Documentación Técnica con LLMs](04_rag_documentacion_datos.ipynb)
- [🔍 Embeddings y Similitud en Datos](05_embeddings_similitud_datos.ipynb) ← 🔵 Estás aquí
- [🤖 Agentes Autónomos para Automatización](06_agentes_automatizacion.ipynb)
- [✅ Validación de Datos con LLMs](07_calidad_validacion_llm.ipynb)
- [🎲 Generación de Datos Sintéticos con LLMs](08_sintesis_aumento_datos.ipynb)
- [🚀 Proyecto Integrador 1: Chatbot de Consulta de Datos con RAG](09_proyecto_integrador_1.ipynb)
- [🏗️ Proyecto Integrador 2: Plataforma Self-Service con GenAI](10_proyecto_integrador_2.ipynb)

**🎓 Otros Niveles:**
- [Nivel Junior](../nivel_junior/README.md)
- [Nivel Mid](../nivel_mid/README.md)
- [Nivel Senior](../nivel_senior/README.md)
- [Nivel GenAI](../nivel_genai/README.md)
- [Negocio LATAM](../negocios_latam/README.md)
